# Bước 2: Basket Preparation for Association Analysis

Notebook này thực hiện bước chuyển đổi dữ liệu giao dịch từ dữ liệu đã làm sạch thành định dạng giỏ hàng (basket format) thích hợp để áp dụng các thuật toán khai thác luật kết hợp như Apriori và FP-Growth trong các bước tiếp theo.

## Mục tiêu

1. Đọc dữ liệu giao dịch đã làm sạch

2. Biến đổi dữ liệu thành ma trận Invoice × Product

3. Mã hoá ma trận thành dạng boolean (0/1)

4. Kiểm tra phân phối số lượng sản phẩm trên mỗi hóa đơn

5. Lưu ma trận boolean ra file để sử dụng cho mô hình mining luật kết hợp


Các thuật toán khai phá luật kết hợp không làm việc với dữ liệu dạng transactional gốc theo từng dòng, mà yêu cầu dữ liệu ở dạng:

        Product A | Product B | Product C | ...
Invoice 1 1 | 0 | 1 | ...
Invoice 2 0 | 1 | 1 | ...
Invoice 3 1 | 1 | 0 | ...

Trong đó:

- `1` nghĩa là sản phẩm có trong giỏ hàng
- `0` nghĩa là không có

Việc chuẩn bị dữ liệu dạng basket là chìa khóa quan trọng trước khi áp dụng Apriori / FP-Growth.


In [1]:
# PARAMETERS (for papermill)

# File dữ liệu đã làm sạch từ bước 1
CLEANED_DATA_PATH = "data/processed/cleaned_uk_data.csv"

# Đường dẫn lưu basket_bool dạng parquet 
BASKET_BOOL_PATH = "data/processed/basket_bool.parquet"

# Tên cột trong dữ liệu đã làm sạch
INVOICE_COL = "InvoiceNo"
ITEM_COL = "Description"
QUANTITY_COL = "Quantity"

# Ngưỡng để coi một item là "có trong giỏ"
# (Quantity >= THRESHOLD -> 1, ngược lại 0)
THRESHOLD = 1


In [2]:
# Parameters
CLEANED_DATA_PATH = "data/processed/cleaned_uk_data.csv"
BASKET_BOOL_PATH = "data/processed/basket_bool.parquet"
INVOICE_COL = "InvoiceNo"
ITEM_COL = "Description"
QUANTITY_COL = "Quantity"
THRESHOLD = 1


## Set up

In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Determine correct project root
cwd = os.getcwd()
if os.path.basename(cwd) == "notebooks":
    project_root = os.path.abspath("..")
else:
    project_root = cwd

src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)


from cluster_library import BasketPreparer


### Tải dữ liệu đã làm sạch

In [4]:
# Đọc dữ liệu đã làm sạch từ Notebook 01
df_clean = pd.read_csv(CLEANED_DATA_PATH, parse_dates=["InvoiceDate"])

print("Thông tin dữ liệu đã làm sạch:")
print(f"- Số giao dịch: {df_clean.shape[0]:,}")
print(f"- Số cột: {df_clean.shape[1]}")
print(f"- Số hoá đơn (InvoiceNo) duy nhất: {df_clean['InvoiceNo'].nunique():,}")
print(f"- Số sản phẩm (Description) duy nhất: {df_clean['Description'].nunique():,}")

df_clean.head()


C:\Users\admin\AppData\Local\Temp\ipykernel_22300\3511701403.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_clean = pd.read_csv(CLEANED_DATA_PATH, parse_dates=["InvoiceDate"])


Thông tin dữ liệu đã làm sạch:
- Số giao dịch: 485,123
- Số cột: 11
- Số hoá đơn (InvoiceNo) duy nhất: 18,021
- Số sản phẩm (Description) duy nhất: 4,007


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,DayOfWeek,HourOfDay
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,017850,United Kingdom,15.30,2,8
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,017850,United Kingdom,20.34,2,8
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,017850,United Kingdom,22.00,2,8
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,017850,United Kingdom,20.34,2,8
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,017850,United Kingdom,20.34,2,8


### Khởi tạo BasketPreparer và tạo basket

In [5]:
# Khởi tạo BasketPreparer
basket_maker = BasketPreparer(
    df=df_clean,
    invoice_col=INVOICE_COL,
    item_col=ITEM_COL,
    quantity_col=QUANTITY_COL,
)

# Tạo basket (Invoice x Item, giá trị = tổng Quantity)
basket = basket_maker.create_basket()

print("Kích thước basket (ma trận Invoice x Item):")
print(f"- Số hoá đơn (rows): {basket.shape[0]:,}")
print(f"- Số sản phẩm (columns): {basket.shape[1]:,}")

basket.iloc[:5, :10]  # xem thử 5 hoá đơn đầu, 10 sản phẩm đầu


Kích thước basket (ma trận Invoice x Item):
- Số hoá đơn (rows): 18,021
- Số sản phẩm (columns): 4,007


Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT
InvoiceNo,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Encode basket thành 0/1

Tỉ lệ ô = 1 càng thấp chứng tỏ ma trận càng thưa (sparse), phù hợp với các thuật toán luật kết hợp.

In [6]:
# Mã hoá basket thành dạng boolean (0/1) với ngưỡng THRESHOLD
basket_bool = basket_maker.encode_basket(threshold=THRESHOLD)

print("Thông tin basket_bool:")
print(f"- Kích thước: {basket_bool.shape[0]:,} x {basket_bool.shape[1]:,}")
print(f"- Tỉ lệ ô = 1 (mua hàng): {basket_bool.values.mean():.4f}")

basket_bool.iloc[:5, :10]


D:\khaiphadulieu\shop_cluster\src\cluster_library.py:269: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_bool = self.basket.applymap(lambda x: 1 if x >= threshold else 0)


Thông tin basket_bool:
- Kích thước: 18,021 x 4,007
- Tỉ lệ ô = 1 (mua hàng): 0.0066


Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT
InvoiceNo,,,,,,,,,,
536365,False,False,False,False,False,False,False,False,False,False
536366,False,False,False,False,False,False,False,False,False,False
536367,False,False,False,False,False,False,False,False,False,False
536368,False,False,False,False,False,False,False,False,False,False
536369,False,False,False,False,False,False,False,False,False,False


In [7]:
# Phân phối số lượng item trong mỗi giỏ (số sản phẩm mỗi hoá đơn)
items_per_invoice = basket_bool.sum(axis=1)

print("Số sản phẩm trung bình trong mỗi hoá đơn:")
print(f"- Mean: {items_per_invoice.mean():.2f}")
print(f"- Median: {items_per_invoice.median():.0f}")
print(f"- Max: {items_per_invoice.max():,}")

items_per_invoice.describe()


Số sản phẩm trung bình trong mỗi hoá đơn:
- Mean: 26.34
- Median: 15
- Max: 1,108


count    18021.000000
mean        26.338549
std         48.846918
min          1.000000
25%          6.000000
50%         15.000000
75%         29.000000
max       1108.000000
dtype: float64

In [8]:
# Lưu basket_bool vào file parquet để dùng cho Apriori ở Notebook 03
basket_maker.save_basket_bool(BASKET_BOOL_PATH)

print("Đã lưu basket_bool thành công:")
print(f"- File: {BASKET_BOOL_PATH}")
print(f"- Kích thước: {basket_bool.shape[0]:,} x {basket_bool.shape[1]:,}")


Đã lưu basket boolean: data/processed/basket_bool.parquet
Đã lưu basket_bool thành công:
- File: data/processed/basket_bool.parquet
- Kích thước: 18,021 x 4,007
